In [1]:
import numpy as np
import pylab as plt
import planet_search as ps
import constants as cs

In [2]:
# just do it
tic = 98796344 # ltt 1445
tic = 318022259 # toi-1730
#tic = 307210830  # l 98-59
ts = ps.run_full_planet_search(tic)

In [145]:
def get_POIs(ts):
    '''
    Given a set of tls result dictionaries (i.e. from multiple TLS runs and/or
    multiple sectors), consolidate periods of interest. 
    '''
    # collect all periods
    POIsv1 = np.zeros((0,2))   # (period, SDE)
    for res in ts.tls.__dict__.keys():
        POIsv1 = np.vstack([POIsv1, [getattr(ts.tls,res).period, getattr(ts.tls,res).SDE_raw]])
    POIsv1 = POIsv1[np.argsort(POIsv1,0)[:,0]]
    
    # identify duplicates
    POIsv2 = np.zeros((0,3))  # avg(P), avg(SDE), Noccurrences
    for p in POIsv1[:,0]:
        #duplicates = np.isclose(POIsv1[:,0], p, rtol=cs.P_duplicate_fraction)
        duplicates = np.isclose(POIsv1[:,0], p, rtol=.02)   # TEMP
        avgP = np.average(POIsv1[duplicates,0], weights=POIsv1[duplicates,1])
        avgSDE = np.average(POIsv1[duplicates,1], weights=POIsv1[duplicates,1])
        Nocc = np.sum(duplicates)
        POIsv2 = np.vstack([POIsv2, [avgP,avgSDE,Nocc]])
    # remove duplicates
    _,inds = np.unique(POIsv2[:,0], return_index=True)
    POIsv3 = POIsv2[inds]
    
    # leave multiples alone because if they surive the iterative TLS, they may be real (e.g. L 98-59)
    ts.star.POIs, ts.star.SDEOIs, ts.star.NoccurrencesOIs = POIsv3.T   # TEMP change star to vetting

In [146]:
get_POIs(ts)

In [165]:
np.ones(3).astype(bool) * np.ones(3).astype(bool)

array([ True,  True,  True])